In [ ]:
%config IPCompleter.greedy=True

import tensorflow as tf
import numpy as np
import cv2
import os
import shutil

from tensorflow import keras
from matplotlib import pyplot as plt

# Conversion du modèle vers un modèle TensorRT

TensorRT réalise plusieurs modifications et optimisations sur le réseau. Tout d'abord, les couches n'utilisant pas de sorties sont supprimées afin d'alléger les calculs. Ensuite, lorsque cela est possible, les couches de convolution, les offsets (bias) et les activations sont fusionnées pour ne former qu'une couche unique.  
Vous pouvez trouver des informations relatives à TensorRT sur le site de Nvidia : https://developer.nvidia.com/tensorrt

Pour réaliser la converison du modèle, nous utilisons l'outil trtexec, en précisant le répertoire dans lequel le modèle est sauvegardé et les paramètres de conversion :  
- **precision_mode** : Format de codage des nombres : FP32, FP16 ou INT8. Les formats inférieurs au FP32 (FP16 et INT8) peuvent améliorer les performances des calculs. Le mode FP16 utilise des coeurs matériels avec des instructions sur des flottants 16bits lorsque cela est possible. Le mode INT8 utilise des coeurs matériels avec des instructions sur des entiers.     
- **max_batch_size** : Le batch-size maximum à utiliser pendant l'optimisation. Pendant l'excéution en temps réel, on peut choisir une valeur plus petite mais pas plus grande.  
- **minimum_segment_size** : Ce paramètre permet de préciser la valeur minimale de noeuds qu'il faut pour que la conversion du réseau soit exécutée. En conséquence, en général on choisit des valeurs inférieures à 5. Ce paramètre permet également de choisir le nombre minimum de noeuds pendant l'optimisation finale INT8 et donc d'optimiser la précision des résultats finaux.  
- **max_workspace_size_byte** : Les opérations d'optimisation de TensorRT ont besoin d'utiliser de l'espace de stockage temporaire. Ce paramètre permet de limiter l'espace maximal utilisé dans le GPU qu'une couche peut utiliser. Si une valeur insuffisante est donnée, TensorRT peut ne pas réussir à optimiser le modèle.

Les arguments disponibles peuvent être obtenus à l'aide de la commande trtexec :

In [ ]:
!/usr/src/tensorrt/bin/trtexec

### Conversion au format FP16

In [ ]:
!/usr/src/tensorrt/bin/trtexec --verbose --fp16 --workspace=256 --maxBatch=1 --onnx="meilleur_model_colab/meilleur_model_colab.onnx" --saveEngine="meilleur_model_colab/meilleur_modele_colab_FP16.engine"